In [80]:
# Library for accessing operation system functionality - environment variables here
import os
# Library used to load environment variables
from dotenv import load_dotenv
# OpenAI library
from openai import OpenAI
# Library to manage tokens
import tiktoken
# DateTime library
from datetime import datetime
# Library to work with JSON data
import json

# Load environment variables from .env file
load_dotenv()
# Access OpenAI's API Key stored in the .env file which is in the root folder
API_KEY = os.getenv('API_KEY')

# Set default values 
DEFAULT_MODEL = 'gpt-4o'
DEFAULT_TEMPERATURE = 0.9
DEFAULT_MAX_TOKENS = 512
DEFAULT_TOKEN_BUDGET = 4096

# Class to manage chatbot conversations 
# Includes session management, token management, persona setting, session management & persistence to files
class ChatManager():
    def __init__(self, api_key=None, model=None, temperature=None,max_tokens=None, token_budget=None,history_filepath=None):
        self.api_key = api_key if api_key else API_KEY
        self.model = model if model else DEFAULT_MODEL
        self.temperature = temperature if temperature else DEFAULT_TEMPERATURE
        self.max_tokens = max_tokens if max_tokens else DEFAULT_MAX_TOKENS
        self.token_budget = token_budget if token_budget else DEFAULT_TOKEN_BUDGET

        # Initiatialize list to track multi-turn conversations - prompts and responses
        self.messages = []
        # Txt file to persist sessions at frequent checkpoints to ensure continuity
        self.history_filepath = history_filepath if history_filepath else self.get_history_filename()
        # Initialize OpenAI client
        self.client = OpenAI(api_key=self.api_key)

    
    def get_history_filepath(self):
        current_dt = datetime.now()
        formatted_dt = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        filename = f"chat_history_{formatted_dt}"
        return filename

    # Function returns JSON messages from file
    def load_json_from_file(self):
      #  print(f'The history_filepath is {self.history_filepath}')
        try:
            with open(self.history_filepath, 'r') as file:
                data = json.load(file)
            return data 
        except FileNotFoundError:
            print(f"Error: The file '{self.history_filepath}' does not exist.")
        except json.JSONDecodeError:
            print(f"Error: The file '{self.history_filepath}' contains invalid JSON.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    # Function prepopulates the messages history list with JSON data from previous conversations 
    def load_conv_history(self):
        json_data = self.load_json_from_file()
        if json_data is not None:
           # print(json_data)
            self.messages = json_data
            
    # Function persists the messages history list to file         
    def save_conv_history(self):
        try:
            with open(self.history_filepath,'w') as file:
                json.dump(self.messages, file, indent=4)
            #print(f'Successfully saved data to {self.history_filepath}')
        except FileNotFoundError:
            print(f'Error: The file {self.history_filepath} doesnt exist')
        except IOError:
            print(f'Error: Could not read file {self.history_filepath}')
        except Exception as e:
            print(f'An unexpected error occurred: {e}')

    def count_tokens(self, text):
        #Initiatize the tokenizer
        tokenizer = tiktoken.encoding_for_model('gpt-4')
        #Tokenize the input text
        tokens = tokenizer.encode(text)
        #return the number of tokens
        return len(tokens)

    def total_tokens_used(self):
        total_tokens = 0
        for row in self.messages:
            total_tokens+=self.count_tokens(row['content'])
        return total_tokens

    def enforce_token_budget(self):
        try:
            while self.total_tokens_used() >= self.token_budget:
                if len(self.messages) > 1:
                    self.messages.pop(1)
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def chat_completion(self,prompt):
        self.load_conv_history()
        #print(f"Total length of messages in conv history is {len(self.messages)}")
        self.messages.append({'role':'user','content':prompt})
        self.enforce_token_budget()
        #print(f"Total tokens used so far is {self.total_tokens_used()}")
        response = self.client.chat.completions.create(seed=122, messages=self.messages,model=self.model, temperature=self.temperature, max_tokens = self.max_tokens)
        self.messages.append({'role':'assistant','content':response.choices[0].message.content})
        self.save_conv_history()
        return response.choices[0].message.content

In [98]:
conv_manager = ChatManager(temperature = 1.0,history_filepath='conv_history.txt')

In [100]:
response = conv_manager.chat_completion(prompt='Create a short funny tag line for Ugg shoe brand')

In [102]:
print(response)

"Ugg: Because your feet deserve a vacation too!"


In [104]:
for row in conv_manager.messages:
    print(f"{row['role']} and {row['content']}")

user and Create a short funny tag line for Ugg shoe brand
assistant and "Ugg: When your feet want a cozy bear hug!"
user and Create a short funny tag line for Ugg shoe brand
assistant and "Ugg: Because your feet deserve a vacation too!"
